<a href="https://colab.research.google.com/github/royaad/AIC/blob/main/%5BSolution%5D_Project_Machine_Translation_Deployment_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Machine Translation Deployment Using Gradio
© 2023, Zaka AI, Inc. All Rights Reserved.

## Importing Libraries

Importing libraries

In [ ]:
import os
# hide TF warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import numpy as np
!pip install -q gradio
import gradio as gr
import string, re, pickle, logging
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.8/288.8 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 4.6 MB/s eta 0:

Cloning git

In [ ]:
#Test Your Zaka
#cloning git repo
!git clone https://github.com/royaad/AIC
#changing working directory
%cd AIC/Week_10/data

Cloning into 'AIC'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 90 (delta 26), reused 7 (delta 3), pack-reused 0
Unpacking objects: 100% (90/90), 7.59 MiB | 2.93 MiB/s, done.
/content/AIC/Week_10/data


##Translator Class

In [ ]:
class Translator:

  def __init__(self, model_directory='./'):
    logging.info("Translator Initialized")
    self.directory = model_directory
    self.model = load_model(model_directory+'EN_FR_Translator.h5')
    logging.info("Model is loaded!")

  def predict(self, in_sentence, in_language='EN', out_language='FR', Max_Length=21):
    in_sentence = self.text_cleaner(in_sentence)
    X = self.Encoder(in_sentence, in_language, Max_Length)
    Y = np.ravel(np.argmax(self.model.predict(X, verbose=0), axis=2))
    out_sentence = self.logits_to_text(Y, out_language)
    out_sentence = out_sentence.replace('<pad>', '')
    return out_sentence.strip()

  def text_cleaner(self, sentence):
    sentence = sentence.lower()
    translation_table = str.maketrans("", "", string.punctuation)
    sentence = re.sub(r"(?<!\d)[- \'](?!\d)", ' ', sentence)
    sentence = sentence.translate(translation_table)
    return sentence

  def Encoder(self, sentence, language, length=21):
    #integer encode sequences
    seq = self.text_to_seq(sentence, language)
    #pad sequences with 0 values
    seq = pad_sequences([seq], maxlen=length, padding='post')
    return seq

  def logits_to_text(self, logits, language):

    word_index = self.word_index_dict(language)
    index_word = dict(zip(word_index.values(), word_index.keys()))
    index_word[0] = '<pad>'

    return ' '.join([index_word[prediction] for prediction in logits])

  def word_index_dict(self, language):
    if language == 'EN':
      pickle_dict = open(self.directory + "EN_dict.pickle","rb")
      dictionary = pickle.load(pickle_dict)
      return dictionary
    elif language == 'FR':
      pickle_dict = open(self.directory + "FR_dict.pickle","rb")
      dictionary = pickle.load(pickle_dict)
      return dictionary
    else:
      print('Input Language Does Not Exist')

  def text_to_seq(self, sentence, language):
    sentence = sentence.split()
    seq = []
    word_index = self.word_index_dict(language)
    #OOVs are dropped
    for x in sentence:
      try:
        seq.append(word_index[x])
      except:
        pass
    return seq

##Translate Function

In [ ]:
#loading model
def translate(sentence):
  model = Translator()
  translation = model.predict(sentence)
  return translation

In [ ]:
input = "She is driving the truck."

translation = translate(input)
print(translation)

elle conduit le camion


##Building Gradio Interphace

In [ ]:
interface = gr.Interface(fn = translate, inputs = gr.components.Textbox(lines=2, placeholder='Enter your English sentence here\n(15 words max)'), outputs = 'text')

In [ ]:
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f63fbd57f30e520550.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
interface.close()

Closing server running on port: 7860
